In [2]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from enum import Enum

import numpy as np
import os
import matplotlib.pyplot as plt
import random
import statistics

from sklearn.preprocessing import scale
from sklearn import decomposition
import pandas as pd

# Pickup_Time_(Hours)
# Pickup_X_Coordinate
# Pickup_Y_Coordinate
# Dropoff_X_Coordinate
# Dropoff_Y_Coordinate
# Passenger_Number
# Tip_Amount

# Load the data into a pandas DataFrame
df = pd.read_csv('./regression.csv')

In [3]:
Pickup_Time_Hours = df["Pickup_Time_(Hours)"].to_list()
min_pt              = min(Pickup_Time_Hours)
max_pt              = max(Pickup_Time_Hours)
diff_pt             = max_pt - min_pt

Pickup_X_Coordinate = df["Pickup_X_Coordinate"].to_list()
min_pxc             = min(Pickup_X_Coordinate)
max_pxc             = max(Pickup_X_Coordinate)
diff_pxc            = max_pxc - min_pxc

Pickup_Y_Coordinate = df["Pickup_Y_Coordinate"].to_list()
min_pyc             = min(Pickup_Y_Coordinate)
max_pyc             = max(Pickup_Y_Coordinate)
diff_pyc            = max_pyc - min_pyc

Dropoff_X_Coordinate = df["Dropoff_X_Coordinate"].to_list()
min_dxc              = min(Dropoff_X_Coordinate)
max_dxc              = max(Dropoff_X_Coordinate)
diff_dxc             = max_dxc - min_dxc

Dropoff_Y_Coordinate = df["Dropoff_Y_Coordinate"].to_list()
min_dyc              = min(Dropoff_Y_Coordinate)
max_dyc              = max(Dropoff_Y_Coordinate)
diff_dyc             = max_dyc - min_dyc

Passenger_Number = df["Passenger_Number"].to_list()
min_pn           = min(Passenger_Number)
max_pn           = max(Passenger_Number)
diff_pn          = max_pn - min_pn

Tip_Amount = df["Tip_Amount"].to_list()
min_ta              = min(Tip_Amount)
max_ta              = max(Tip_Amount)
diff_ta          = max_ta - min_ta

In [4]:
Pickup_Time_Hours = [(pt-min_pt)/diff_pt * 10 for pt in Pickup_Time_Hours]
Pickup_X_Coordinate = [(pt-min_pxc)/diff_pxc * 10 for pt in Pickup_X_Coordinate]
Pickup_Y_Coordinate = [(pt-min_pyc)/diff_pyc * 10 for pt in Pickup_Y_Coordinate]
Dropoff_X_Coordinate = [(pt-min_dxc)/diff_dxc * 10 for pt in Dropoff_X_Coordinate]
Dropoff_Y_Coordinate = [(pt-min_dyc)/diff_dyc * 10 for pt in Dropoff_Y_Coordinate]
Passenger_Number = [(pt-min_pn)/diff_pn * 10 for pt in Passenger_Number]
Tip_Amount = [(pt-min_ta)/diff_ta * 10 for pt in Tip_Amount]

In [5]:
df["Pickup_Time_(Hours)"] = Pickup_Time_Hours
df["Pickup_X_Coordinate"] = Pickup_X_Coordinate
df["Pickup_Y_Coordinate"] = Pickup_Y_Coordinate
df["Dropoff_X_Coordinate"] = Dropoff_X_Coordinate
df["Dropoff_Y_Coordinate"] = Dropoff_Y_Coordinate
df["Passenger_Number"] = Passenger_Number
df["Tip_Amount"] = Tip_Amount

In [6]:
# Split the data into training and testing sets
X = df.drop(["Tip_Amount"], axis=1)
y = df["Tip_Amount"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
# Convert the data into DMatrix format, which is an optimized data structure for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set the hyperparameters for the XGBoost model
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 5,
    'alpha': 0.1,
    'lambda': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

# Train the XGBoost model
model = xgb.train(params, dtrain, num_boost_round=100)

# Predict on the test set
y_pred = model.predict(dtest)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)
print("The error rate is: ", mse /10 *100, "%")

Mean Squared Error: 0.8310951373293983
The error rate is:  8.310951373293983 %


In [8]:
# Split the data into training and testing sets
X = df["Tip_Amount"]
y1 = df["Dropoff_X_Coordinate"]
y2 = df["Dropoff_Y_Coordinate"]
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2)
_, _, y2_train, y2_test = train_test_split(X, y2, test_size=0.2)

# Convert the data into DMatrix format, which is an optimized data structure for XGBoost
dtrain1 = xgb.DMatrix(X_train, label=y1_train)
dtest1 = xgb.DMatrix(X_test, label=y1_test)
dtrain2 = xgb.DMatrix(X_train, label=y2_train)
dtest2 = xgb.DMatrix(X_test, label=y2_test)

# Set the hyperparameters for the XGBoost model
params1 = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 5,
    'alpha': 0.1,
    'lambda': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

params2 = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 5,
    'alpha': 0.1,
    'lambda': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

# Train the XGBoost models
model1 = xgb.train(params1, dtrain1, num_boost_round=100)
model2 = xgb.train(params2, dtrain2, num_boost_round=100)

# Predict on the test set
y1_pred = model1.predict(dtest1)
y2_pred = model2.predict(dtest2)

# Calculate the mean squared errors
res = 0
for i in range(len(y1_pred)):
    res += abs(y1_test.tolist()[i]-y1_pred.tolist()[i]) + abs(y2_test.tolist()[i]-y2_pred.tolist()[i])
error = res / len(y1_pred)
print('Overall Manhattan distance error is:', error)

Overall Manhattan distance error is: 3.151675549773707


In [ ]:
epsilon=10

regression_array=[]
with open('regression.csv', newline='') as pre:
    reader = csv.reader(pre, delimiter=',')
    line = 0
    for row in reader:
        if line == 0:
            data_rep = row
        else:
            regression_array.append(row)
        line += 1

num_data=len(regression_array)
num_col=len(regression_array[0])

dif=np.zeros(num_col)
Min=np.zeros(num_col)
Max=np.zeros(num_col)
for i in range(num_data):
    if (i==0):
        for j in range(num_col):
            Max[j]=regression_array[i][j]
            Min[j]=regression_array[i][j]
    else:
        for j in range(num_col):
            if (float(regression_array[i][j])>Max[j]):
                Max[j]=regression_array[i][j]
            if (float(regression_array[i][j])<Min[j]):
                Min[j]=regression_array[i][j]
dif=Max-Min
randomV=[]
for i in range(num_col):
    randomV.append(laplace_draw(0,dif[i]/epsilon,Min[i],Max[i],num_data))

for i in range(num_data):
    for j in range(num_col):
        regression_array[i][j]=max(0,float(regression_array[i][j])+randomV[j][i])